# Io Persona v1.3 + ICP v1.0 — Installer Notebook

This notebook helps you:

1. Load the Io v1.3 persona config and Io–Ceven Collaborative Protocol (ICP v1.0)
2. Build a combined `system` prompt
3. (Optionally) test a call against an OpenAI-compatible API, such as OpenAI or LM Studio

## File Expectations

By default, this notebook expects the following files to be in the **same directory**:

- `Io_v1_3_Config.json`
- `Io_v1_3_Implementable_Spec.md` (optional, for reference)
- `ICP_v1_0.md`
- `ICP_v1_0.json` (optional, for structured reference)

You can adjust the paths in the code cells if you place them elsewhere.

## 1. Load Configs and Build System Prompt

In [ ]:
from pathlib import Path
import json

BASE = Path('.')  # adjust if your config files live elsewhere

io_cfg_path = BASE / 'Io_v1_3_Config.json'
icp_md_path = BASE / 'ICP_v1_0.md'

with io_cfg_path.open('r', encoding='utf-8') as f:
    io_cfg = json.load(f)

with icp_md_path.open('r', encoding='utf-8') as f:
    icp_text = f.read()

persona_prompt = io_cfg.get('system_prompt', '').strip()
if not persona_prompt:
    raise ValueError("Io config is missing 'system_prompt'.")

combined_system_prompt = (
    persona_prompt
    + '\n\n---\n\n'
    + 'The following collaboration protocol with the user **Ceven** is in effect. '
      'You must align your behavior to it while preserving safety policies:'
    + '\n\n'
    + icp_text.strip()
)

print(combined_system_prompt[:1000])  # preview first ~1000 characters

## 2. (Optional) Test with an OpenAI-Compatible Backend

This cell assumes you have installed the `openai` Python package and have an API-compatible
backend configured (OpenAI, LM Studio, etc.). Adjust the `base_url` and `model` as needed.

If you are using LM Studio, you typically set `base_url` to something like
`'http://localhost:1234/v1'` and provide a dummy `OPENAI_API_KEY`.

In [ ]:
from openai import OpenAI

# Configure your backend here
MODEL = 'gpt-4.1-mini'  # or your local model id
BASE_URL = None  # e.g. 'http://localhost:1234/v1' for LM Studio

client_kwargs = {}
if BASE_URL is not None:
    client_kwargs['base_url'] = BASE_URL

client = OpenAI(**client_kwargs)

response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": combined_system_prompt},
        {"role": "user", "content": "Hey Io, please confirm that you are running with Io v1.3 + ICP v1.0."},
    ],
)
print(response.choices[0].message.content)

## 3. Next Steps

- Commit this notebook and the config files into your GitHub repo using the suggested structure.
- Use the combined system prompt in any LLM client where you want to instantiate Io.
- Optionally, create additional notebooks that:
  - Test specific modes (Executor / Explorer / Challenger / Synthesizer / Visionary)
  - Benchmark responses across different models or providers
  - Log and review Io's behavior for consistency with the Io v1.3 Blueprint and ICP v1.0.